In [2]:
import pandas as pd
import numpy as np

data= pd.read_csv('global-data-on-sustainable-energy.csv')
missing_values = data.isnull().sum()
length=len(data)
missing_percentage = (missing_values / len(data)) * 100
print(f"Missing values are  :{missing_values}")
print(f"Missing percentages are :{missing_percentage}")

Missing values are  :Entity                                                                 0
Year                                                                   0
Access to electricity (% of population)                               10
Access to clean fuels for cooking                                    169
Renewable-electricity-generating-capacity-per-capita                 931
Financial flows to developing countries (US $)                      2089
Renewable energy share in the total final energy consumption (%)     194
Electricity from fossil fuels (TWh)                                   21
Electricity from nuclear (TWh)                                       126
Electricity from renewables (TWh)                                     21
Low-carbon electricity (% electricity)                                42
Primary energy consumption per capita (kWh/person)                     0
Energy intensity level of primary energy (MJ/$2017 PPP GDP)          207
Value_co2_emissions_kt_by_coun

In [3]:
data_types = data.dtypes
print(data_types)

Entity                                                               object
Year                                                                  int64
Access to electricity (% of population)                             float64
Access to clean fuels for cooking                                   float64
Renewable-electricity-generating-capacity-per-capita                float64
Financial flows to developing countries (US $)                      float64
Renewable energy share in the total final energy consumption (%)    float64
Electricity from fossil fuels (TWh)                                 float64
Electricity from nuclear (TWh)                                      float64
Electricity from renewables (TWh)                                   float64
Low-carbon electricity (% electricity)                              float64
Primary energy consumption per capita (kWh/person)                  float64
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         float64
Value_co2_em

In [4]:
data_types = data.dtypes
print(data_types)

Entity                                                               object
Year                                                                  int64
Access to electricity (% of population)                             float64
Access to clean fuels for cooking                                   float64
Renewable-electricity-generating-capacity-per-capita                float64
Financial flows to developing countries (US $)                      float64
Renewable energy share in the total final energy consumption (%)    float64
Electricity from fossil fuels (TWh)                                 float64
Electricity from nuclear (TWh)                                      float64
Electricity from renewables (TWh)                                   float64
Low-carbon electricity (% electricity)                              float64
Primary energy consumption per capita (kWh/person)                  float64
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         float64
Value_co2_em

In [5]:
def remove_functuation(density):
    return density.replace(',','')
data['Entity']=data['Entity'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].apply(remove_functuation)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(float)

In [6]:
# dropping ALbania as there is no good data available
indices_to_drop = data[data['Entity'] == 'Albania'].index
data.drop(indices_to_drop, inplace=True)

In [7]:
columns=data.columns.tolist()
feature_with_null=[column for column in columns if column not in ['Entity','Year','Primary energy consumption per capita (kWh/person)']]
feature_with_null

['Access to electricity (% of population)',
 'Access to clean fuels for cooking',
 'Renewable-electricity-generating-capacity-per-capita',
 'Financial flows to developing countries (US $)',
 'Renewable energy share in the total final energy consumption (%)',
 'Electricity from fossil fuels (TWh)',
 'Electricity from nuclear (TWh)',
 'Electricity from renewables (TWh)',
 'Low-carbon electricity (% electricity)',
 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
 'Value_co2_emissions_kt_by_country',
 'Renewables (% equivalent primary energy)',
 'gdp_growth',
 'gdp_per_capita',
 'Density\\n(P/Km2)',
 'Land Area(Km2)',
 'Latitude',
 'Longitude']

In [8]:
data.replace({None: np.nan}, inplace=True)

columns=['Financial flows to developing countries (US $)','Renewable-electricity-generating-capacity-per-capita','Renewables (% equivalent primary energy)']
data.drop(columns=columns,inplace=True)

columns=data.columns.tolist()
feature_with_null=[column for column in columns if column not in ['Entity','Year','Primary energy consumption per capita (kWh/person)']]

for column in feature_with_null:
    # Group by 'Country_Encoded' and calculate the mean for each group
    means = data.groupby('Entity')[column].transform('mean')

    # If the mean is NaN, replace it with the global mean of the column
    global_mean = data[column].mean()
    means_filled = means.fillna(global_mean)

    # Fill the missing values with the calculated means
    data[column] = data[column].fillna(means_filled)

zero_columns=['Electricity from nuclear (TWh)','Electricity from renewables (TWh)','Low-carbon electricity (% electricity)']
data.drop(columns=zero_columns,inplace=True, axis=1)




In [9]:
zero_counts=(data==0).sum()
print(zero_counts)

Entity                                                                0
Year                                                                  0
Access to electricity (% of population)                               0
Access to clean fuels for cooking                                     8
Renewable energy share in the total final energy consumption (%)     97
Electricity from fossil fuels (TWh)                                 130
Primary energy consumption per capita (kWh/person)                   21
Energy intensity level of primary energy (MJ/$2017 PPP GDP)           0
Value_co2_emissions_kt_by_country                                     0
gdp_growth                                                            1
gdp_per_capita                                                        0
Density\n(P/Km2)                                                      0
Land Area(Km2)                                                        0
Latitude                                                        

In [10]:
columns=data.columns.to_list()

df=data

def rmOutliers(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# List of columns to check for outliers
columns_to_check = [
    'Access to electricity (% of population)',
 'Access to clean fuels for cooking',
 'Renewable energy share in the total final energy consumption (%)',
 'Electricity from fossil fuels (TWh)',
 'Primary energy consumption per capita (kWh/person)',
 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
 'Value_co2_emissions_kt_by_country',
 'gdp_growth',
 'gdp_per_capita',
 'Density\\n(P/Km2)',
 'Land Area(Km2)',
 'Latitude',
 'Longitude']

print (columns_to_check)

# Apply the function to each group of entities
df_cleaned = df.groupby('Entity').apply(lambda x: rmOutliers(x, columns_to_check)).reset_index(drop=True)

['Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Renewable energy share in the total final energy consumption (%)', 'Electricity from fossil fuels (TWh)', 'Primary energy consumption per capita (kWh/person)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'Value_co2_emissions_kt_by_country', 'gdp_growth', 'gdp_per_capita', 'Density\\n(P/Km2)', 'Land Area(Km2)', 'Latitude', 'Longitude']


In [11]:
data=df_cleaned
print(f"Total Outliers Removed considering for each country are:{df.count()-df_cleaned.count()}")

Total Outliers Removed considering for each country are:Entity                                                              792
Year                                                                792
Access to electricity (% of population)                             792
Access to clean fuels for cooking                                   792
Renewable energy share in the total final energy consumption (%)    792
Electricity from fossil fuels (TWh)                                 792
Primary energy consumption per capita (kWh/person)                  792
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         792
Value_co2_emissions_kt_by_country                                   792
gdp_growth                                                          792
gdp_per_capita                                                      792
Density\n(P/Km2)                                                    792
Land Area(Km2)                                                      792
Latitude

## data quality plan
### Issues found:
**High missing value rate**: Most columns have missing values. Columns like **Financial flows to developing countries (US $)** and **Renewables (% equivalent primary energy)** have missing rate over 50%.</p>
**Type inconsistency**: Type of the columns like **Entity** and **Density\n(P/Km2)** have type of Object. They should be string and float instead. And **Density\n(P/Km2)** has commas in the value which makes it can not be processed directly.</p>
**Outliers**: The dataset has outliers which can cause poor performance in model.
### Solution
We imputed the missing values with mean values. It won't be wise to drop because we will lose so  many data points. And for type conversion, **Entity** is converted to string. For **Density\n(P/Km2)**, it was firstly converted to string to remove the commas. Then it is converted to float. As for outliers, we implemented a method to remove all values below lower bound and above the upper bound which were determined by Interquartile range of each column.

In [12]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import joblib


label_encoder=LabelEncoder()
data['Entity']=label_encoder.fit_transform(data['Entity'])

target_variables = ['Value_co2_emissions_kt_by_country', 'Renewable energy share in the total final energy consumption (%)']

models={
    'Linear Regression':LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Support Vector Regressor':SVR(),
    'Random Forest Regressor':RandomForestRegressor()
}

def evaluate_model_with_base_parameters(data, models_dict):
    random_forest_features = []
    for i in range(len(target_variables)):
        print(f"Below are the metrics for the target variable: {target_variables[i]}")
        for name, model in models_dict.items():
            if name == 'Support Vector Regressor':
                # Skipping RFE for SVR as it does not have the feature_importances_ or coef_ attribute
                correlation_matrix = data.select_dtypes("number").corr()
                target_features = correlation_matrix[target_variables[i]].drop(target_variables[i])
                target_features = target_features[abs(target_features) > 0.3].index.tolist()
                X = data.drop(columns=target_features)
                y = data[target_variables[i]]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

            else:
                # Applying RFE for other models
                rfe = RFE(estimator=model, n_features_to_select=10)
                X = data.drop(columns=target_variables)
                y = data[target_variables[i]]
                fit = rfe.fit(X, y)
                feature_ranking = pd.DataFrame({'Feature': X.columns, 'Ranking': fit.ranking_}).sort_values(by='Ranking')
                final_features = feature_ranking[feature_ranking['Ranking'] == 1]['Feature'].to_list()
                if 'Entity'  not in final_features:
                    final_features.append('Entity')
                if 'Year' not in final_features:
                  final_features.append('Year')
                print(f"Feature importances for {name} are {final_features}")
                X = X[final_features]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
                model.fit(X_train, y_train)
                if name == 'Random Forest Regressor':
                    model_filename = f'Random forest regressor model for {target_variables[i]}'
                    joblib.dump(model, model_filename)
                    random_forest_features.append(final_features)  # Capture the features for Random Forest Regressor
                y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            print(f"Model: {name}")
            print(f"Mean squared error is: {mse}")
            print(f"Mean absolute error is: {mae}")
            print(f"R2 score error is: {r2}")
            print("\n")

    return random_forest_features


random_forest_features=evaluate_model_with_base_parameters(data, models)


Below are the metrics for the target variable: Value_co2_emissions_kt_by_country
Feature importances for Linear Regression are ['Entity', 'Year', 'Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Electricity from fossil fuels (TWh)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'gdp_growth', 'Density\\n(P/Km2)', 'Latitude', 'Longitude']
Model: Linear Regression
Mean squared error is: 42353824007.1602
Mean absolute error is: 59258.006875401596
R2 score error is: 0.9573043498652017


Feature importances for Decision Tree Regression are ['Entity', 'Year', 'Access to electricity (% of population)', 'Electricity from fossil fuels (TWh)', 'Primary energy consumption per capita (kWh/person)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'gdp_growth', 'gdp_per_capita', 'Land Area(Km2)', 'Longitude']
Model: Decision Tree Regression
Mean squared error is: 47533351744.33949
Mean absolute error is: 18979.780632180547
R2 score error

In [13]:
random_forest_features

[['Year',
  'Access to clean fuels for cooking',
  'Electricity from fossil fuels (TWh)',
  'Primary energy consumption per capita (kWh/person)',
  'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
  'gdp_growth',
  'gdp_per_capita',
  'Density\\n(P/Km2)',
  'Land Area(Km2)',
  'Longitude',
  'Entity'],
 ['Entity',
  'Access to electricity (% of population)',
  'Access to clean fuels for cooking',
  'Electricity from fossil fuels (TWh)',
  'Primary energy consumption per capita (kWh/person)',
  'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
  'Density\\n(P/Km2)',
  'Land Area(Km2)',
  'Latitude',
  'Longitude',
  'Year']]

In [14]:
!pip install prophet

In [15]:
# doing the time series forecasting

from prophet import Prophet

In [16]:
# decoding the label encoded columns

# saving the mapping of the label encoders

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping['Afghanistan'])

data['Country_Decoded'] = label_encoder.inverse_transform(data['Entity'])
data.head()

0


,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude,Country_Decoded
0,0,2000,1.613591,6.2,44.99,0.16,302.59482,1.64,760.000000,6.163893,439.055765,60.0,652230.0,33.93911,67.709953,Afghanistan
1,0,2001,4.074574,7.2,45.60,0.09,236.89185,1.74,730.000000,6.163893,439.055765,60.0,652230.0,33.93911,67.709953,Afghanistan
2,0,2008,42.400000,16.7,21.17,0.19,607.83350,1.94,3559.999943,3.924984,364.663542,60.0,652230.0,33.93911,67.709953,Afghanistan
3,0,2011,43.222020,21.8,12.61,0.18,1436.11430,3.23,8930.000305,0.426355,591.190030,60.0,652230.0,33.93911,67.709953,Afghanistan
4,0,2013,68.290650,24.8,16.86,0.22,1060.79260,2.46,5989.999771,5.600745,624.315454,60.0,652230.0,33.93911,67.709953,Afghanistan


In [17]:
# function to filter the data based on the country based on the user input

def filter_data_by_country(data, country):
  return data[data['Entity']==label_mapping[country]]

In [18]:
# Assuming random_forest_features is defined somewhere in your code
co2_emission_features = random_forest_features[0].copy()
renewable_energy_percentage_features = random_forest_features[1].copy()
features_list = [co2_emission_features, renewable_energy_percentage_features]

# Assuming data_1 is defined somewhere in your code
data_1 = data

for column in features_list:
  column.remove('Year')
  column.remove('Entity')

In [19]:
def predict_all_variables(country, data, input_year):
    pred_dict = {"co2_emission": {}, "renewable_energy_percentage": {}}
    feature_dict_keys = ["co2_emission", "renewable_energy_percentage"]

    for idx, feature_set in enumerate(features_list):
        for column in feature_set:
            country_data = data_1[data_1['Country_Decoded'] == country][['Year', column]].copy()
            country_data['ds'] = pd.to_datetime(country_data['Year'], format='%Y')
            country_data['y'] = country_data[column]
            country_data = country_data[['ds', 'y']]

            m = Prophet()
            m.fit(country_data)

            future = m.make_future_dataframe(periods=15, freq='Y')
            forecast = m.predict(future)
            forecast['ds'] = pd.to_datetime(forecast['ds'])
            forecast['year'] = forecast['ds'].dt.year

            matching_row = forecast[forecast['year'] == input_year]
            if not matching_row.empty:
                pred_dict[feature_dict_keys[idx]][column] = matching_row['yhat'].values[0]
            else:
                pred_dict[feature_dict_keys[idx]][column] = None  # Handle the case where there's no matching row

    return pred_dict

In [20]:
predicted_values = predict_all_variables('Afghanistan', data_1, 2022 )
input_data=predicted_values

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 8.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvwojv3m9/0aw7yqb3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvwojv3m9/hutyf51r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28127', 'data', 'file=/tmp/tmpvwojv3m9/0aw7yqb3.json', 'init=/tmp/tmpvwojv3m9/hutyf51r.json', 'output', 'file=/tmp/tmpvwojv3m9/prophet_modelu0ifk9pw/prophet_model-20240724194503.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:45:03 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:45:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:c

In [21]:
predicted_values

{'co2_emission': {'Access to clean fuels for cooking': 37.00491834890717,
  'Electricity from fossil fuels (TWh)': 0.16055347320412916,
  'Primary energy consumption per capita (kWh/person)': 918.6429215663597,
  'Energy intensity level of primary energy (MJ/$2017 PPP GDP)': 2.031556391662323,
  'gdp_growth': -2.733936660319392,
  'gdp_per_capita': 427.6447736682827,
  'Density\\n(P/Km2)': 60.0,
  'Land Area(Km2)': 652230.0,
  'Longitude': 67.709953},
 'renewable_energy_percentage': {'Access to electricity (% of population)': 122.4576463670294,
  'Access to clean fuels for cooking': 37.00491834890717,
  'Electricity from fossil fuels (TWh)': 0.16055347320412916,
  'Primary energy consumption per capita (kWh/person)': 918.6429215663597,
  'Energy intensity level of primary energy (MJ/$2017 PPP GDP)': 2.031556391662323,
  'Density\\n(P/Km2)': 60.0,
  'Land Area(Km2)': 652230.0,
  'Latitude': 33.93911,
  'Longitude': 67.709953}}

In [22]:
extra_columns=['co2_emission','renewable_energy_percentage']

In [23]:
for column in extra_columns:
   input_data[column]['Year']=2022
   input_data[column]['Entity']=label_mapping['Afghanistan']

In [24]:
input_data

{'co2_emission': {'Access to clean fuels for cooking': 37.00491834890717,
  'Electricity from fossil fuels (TWh)': 0.16055347320412916,
  'Primary energy consumption per capita (kWh/person)': 918.6429215663597,
  'Energy intensity level of primary energy (MJ/$2017 PPP GDP)': 2.031556391662323,
  'gdp_growth': -2.733936660319392,
  'gdp_per_capita': 427.6447736682827,
  'Density\\n(P/Km2)': 60.0,
  'Land Area(Km2)': 652230.0,
  'Longitude': 67.709953,
  'Year': 2022,
  'Entity': 0},
 'renewable_energy_percentage': {'Access to electricity (% of population)': 122.4576463670294,
  'Access to clean fuels for cooking': 37.00491834890717,
  'Electricity from fossil fuels (TWh)': 0.16055347320412916,
  'Primary energy consumption per capita (kWh/person)': 918.6429215663597,
  'Energy intensity level of primary energy (MJ/$2017 PPP GDP)': 2.031556391662323,
  'Density\\n(P/Km2)': 60.0,
  'Land Area(Km2)': 652230.0,
  'Latitude': 33.93911,
  'Longitude': 67.709953,
  'Year': 2022,
  'Entity': 0}

In [25]:
loaded_model_list=['./Random forest regressor model for Value_co2_emissions_kt_by_country', './Random forest regressor model for Renewable energy share in the total final energy consumption (%)']

In [26]:
# Convert each inner dictionary to a DataFrame
df_co2_emission = pd.DataFrame([input_data['co2_emission']])
df_renewable_energy_percentage = pd.DataFrame([input_data['renewable_energy_percentage']])

model_1=joblib.load(loaded_model_list[0])
model_2=joblib.load(loaded_model_list[1])

df_co2_emission=df_co2_emission[random_forest_features[0]]
predicted_value_1 = model_1.predict(df_co2_emission)
print(f"The predicted value of the co_2 emission  is: {predicted_value_1[0]}")

df_renewable_energy_percentage=df_renewable_energy_percentage[random_forest_features[1]]
predicted_value_2 = model_2.predict(df_renewable_energy_percentage)
print(f"The predicted value of the renewable energy consumption  is: {predicted_value_2[0]}")

The predicted value of the co_2 emission  is: 5565.235060948002
The predicted value of the renewable energy consumption  is: 22.388129999999997


# Neural Network Training

In [27]:
!pip install tensorflow

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
data.columns.to_list()

['Entity',
 'Year',
 'Access to electricity (% of population)',
 'Access to clean fuels for cooking',
 'Renewable energy share in the total final energy consumption (%)',
 'Electricity from fossil fuels (TWh)',
 'Primary energy consumption per capita (kWh/person)',
 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
 'Value_co2_emissions_kt_by_country',
 'gdp_growth',
 'gdp_per_capita',
 'Density\\n(P/Km2)',
 'Land Area(Km2)',
 'Latitude',
 'Longitude',
 'Country_Decoded']

In [30]:
target_variables=['Value_co2_emissions_kt_by_country','Renewable energy share in the total final energy consumption (%)']

In [31]:
correlation_matrix =  data.select_dtypes("number").corr()
target_features = correlation_matrix[target_variables[0]].drop(target_variables[0])
target_features = target_features[abs(target_features)>0.3].index.tolist()

In [32]:
target_features

['Electricity from fossil fuels (TWh)', 'Land Area(Km2)']

In [33]:
target_variables

['Value_co2_emissions_kt_by_country',
 'Renewable energy share in the total final energy consumption (%)']

In [34]:
from sklearn.preprocessing import StandardScaler

columns=['Electricity from fossil fuels (TWh)', 'Land Area(Km2)','Value_co2_emissions_kt_by_country','Renewable energy share in the total final energy consumption (%)']

scaler=StandardScaler()
data[columns]=scaler.fit_transform(data[columns])

In [35]:
data[target_variables].head()

,Value_co2_emissions_kt_by_country,Renewable energy share in the total final energy consumption (%)
0,-0.211297,0.426790
1,-0.211334,0.447388
2,-0.207841,-0.377521
3,-0.201214,-0.666560
4,-0.204842,-0.523053


In [36]:
from sklearn.model_selection import train_test_split

X=data[target_features]
y=data[target_variables[0]]

X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=42, test_size=0.2)

In [37]:
neural_network=keras.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

In [38]:
neural_network.compile(optimizer='adam', loss='mean_squared_error')

In [39]:
neural_network.fit(X_train, y_train, epochs=40, batch_size=32)

Epoch 1/40
71/71 [==============================] - 4s 5ms/step - loss: 0.6222
Epoch 2/40
71/71 [==============================] - 0s 4ms/step - loss: 0.3091
Epoch 3/40
71/71 [==============================] - 0s 4ms/step - loss: 0.1599
Epoch 4/40
71/71 [==============================] - 0s 5ms/step - loss: 0.0787
Epoch 5/40
71/71 [==============================] - 0s 5ms/step - loss: 0.0455
Epoch 6/40
71/71 [==============================] - 1s 11ms/step - loss: 0.0361
Epoch 7/40
71/71 [==============================] - 0s 5ms/step - loss: 0.0328
Epoch 8/40
71/71 [==============================] - 0s 4ms/step - loss: 0.0299
Epoch 9/40
71/71 [==============================] - 0s 7ms/step - loss: 0.0292
Epoch 10/40
71/71 [==============================] - 1s 7ms/step - loss: 0.0283
Epoch 11/40
71/71 [==============================] - 1s 8ms/step - loss: 0.0280
Epoch 12/40
71/71 [==============================] - 0s 6ms/step - loss: 0.0283
Epoch 13/40
71/71 [=============================

In [40]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred=neural_network.predict(X_test)

# calculating the scores based on the performance of the neural network

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print(f"Mean squared error (same as neural_network.evaluate()) is: {mse}")
print(f"Mean absolute error is: {mae}")
print(f"R2 score error is: {r2}")

18/18 [==============================] - 0s 1ms/step
Mean squared error (same as neural_network.evaluate()) is: 0.017032467871604427
Mean absolute error is: 0.04510691702510425
R2 score error is: 0.9811151789435855


In [41]:
test_loss = neural_network.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")

18/18 [==============================] - 0s 2ms/step - loss: 0.0170
Test Loss: 0.0170
